In [ ]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as dist
import numpy as np
import tqdm

In [2]:
class Net(nn.Module):
    def __init__(self, input_dim=4, hid_dim=128, nlayers=2, out_dim=2):
        super(Net, self).__init__()
        self.model = []
        self.out_dim = out_dim
        inp = input_dim
        for i in range(nlayers):
            self.model.append(nn.Linear(inp, hid_dim))
            self.model.append(nn.ReLU())
            inp = hid_dim
        self._pi = nn.Linear(inp, out_dim)
        self._v = nn.Linear(inp, 1)
        self.model = nn.Sequential(*self.model)
    
    def pi(self, x):
        out = self.model(x)
        return self._pi(out)
    
    def v(self, x):
        out = self.model(x)
        return self._v(out)

In [3]:
class PPO:
    def __init__(self, model, optim, gamma, lambd, eps):
        self.model = model
        self.optim = optim
        self.gamma = gamma
        self.lambd = lambd
        self.eps = eps
    
    def learn(self, data):
        s = torch.from_numpy(np.array(data['state'])).float()
        a = torch.from_numpy(np.array(data['action'])).long()[:, None]
        r = torch.from_numpy(np.array(data['reward'])).float()[:, None]
        ns = torch.from_numpy(np.array(data['next_state'])).float()
        d = torch.from_numpy(np.array(data['done'])).float()[:, None]
        prob = torch.from_numpy(np.array(data['prob'])).float()[:, None]
        
        for epoch in range(10):
            td_target = r + self.gamma * self.model.v(ns) * d
            delta = td_target - self.model.v(s)
            delta = delta.detach().numpy()

            advantage = []
            A = 0
            for delta_t in delta[::-1]:
                A = self.gamma * self.lambd * A + delta_t[0]
                advantage.append(A)
            advantage = torch.FloatTensor(advantage[::-1])[:, None]
            
            advantage = (advantage - advantage.mean()) / (advantage.std() + 1e-10)

            pi_a = torch.softmax(self.model.pi(s), dim=1).gather(1, a)
            ratio = pi_a / (prob + 1e-10)

            obj = ratio * advantage
            obj_clamp = torch.clamp(ratio, 1-self.eps, 1+self.eps) * advantage
            loss = -torch.min(obj, obj_clamp).mean() + F.mse_loss(self.model.v(s), td_target.detach())

            self.optim.zero_grad()
            loss.backward()
            self.optim.step()
    
    def sample_action(self, state):
        state = torch.from_numpy(state).float()
        prob = torch.softmax(self.model.pi(state), dim=0)
        m = dist.Categorical(prob)
        action = m.sample().item()
        return action, prob[action].item()

In [4]:
class Args:
    lr = 5e-4
    gamma = 0.98
    lambd = 0.95
    eps = 0.1
    max_episodes = 1000

In [5]:
args = Args

In [ ]:
all_score_list = []
for k in range(10):
    env = gym.make('CartPole-v1')
    lr = args.lr
    gamma = args.gamma
    lambd = args.lambd
    eps = args.eps
    max_episodes = args.max_episodes

    n_obs = env.observation_space.shape[0]
    n_act = env.action_space.n
    model = Net(n_obs, hid_dim=128, nlayers=2, out_dim=n_act)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    policy = PPO(model, optimizer, gamma, lambd, eps)

    score_list = []
    for i in tqdm.tqdm(range(max_episodes)):
        s = env.reset()
        data_buffer = {'state': [], 'reward': [], 'action': [], 'next_state': [], 'done': [], 'prob': []}
        done = False
        while not done:
            a, prob = policy.sample_action(s)
            next_s, r, done, info = env.step(a)
            data_buffer['state'].append(s)
            data_buffer['reward'].append(r/100)
            data_buffer['action'].append(a)
            data_buffer['done'].append(0 if done else 1)
            data_buffer['next_state'].append(next_s)
            data_buffer['prob'].append(prob)
            s = next_s

        policy.learn(data_buffer)

        score_list.append(sum(data_buffer['reward'])*100)
    all_score_list.append(score_list)